In [5]:
import os

import psycopg
import pandas as pd
import mlflow
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.environ["DB_DESTINATION_HOST"],
    "port": os.environ["DB_DESTINATION_PORT"],
    "dbname": os.environ["DB_DESTINATION_NAME"],
    "user": os.environ["DB_DESTINATION_USER"],
    "password": os.environ["DB_DESTINATION_PASSWORD"],
}

connection.update(postgres_credentials)


TABLE_NAME = "users_churn"
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "churn_nikolaistepanov_myown"
RUN_NAME = "feature_selection"
REGISTRY_MODEL_NAME = "hp_tuning_model" 
FS_ASSETS = "fs_assets"


with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

yes_no_map = {"Yes": 1, "No": 0}

for col in df.columns:
    if df[col].dtype == "object":
        unique_vals = set(df[col].dropna().unique())
        if unique_vals.issubset({"Yes", "No"}):
            df[col] = df[col].map(yes_no_map)


df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,17,8191-XWSZG,2015-10-01,NaT,One year,0,Mailed check,20.65,1022.95,None,...,NaN,NaN,NaN,NaN,Female,0,0,0,0.0,0
1,59,3957-SQXML,2017-04-01,NaT,Two year,0,Credit card (automatic),24.95,894.30,None,...,NaN,NaN,NaN,NaN,Female,0,1,1,1.0,0
2,147,6837-BJYDQ,2019-11-01,NaT,One year,0,Mailed check,19.60,61.35,None,...,NaN,NaN,NaN,NaN,Male,0,0,0,0.0,0
3,481,0486-LGCCH,2019-03-01,NaT,Two year,0,Mailed check,19.65,225.75,None,...,NaN,NaN,NaN,NaN,Male,0,1,1,0.0,0
4,1001,8357-EQXFO,2019-04-01,2019-11-01,Month-to-month,1,Electronic check,95.35,660.90,Fiber optic,...,1.0,0.0,1.0,1.0,Female,0,0,0,0.0,1


In [6]:
features = ["monthly_charges", "total_charges", "senior_citizen"]
target = "target"

split_column = "begin_date"
stratify_column = "target"
test_size = 0.2

df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=test_size, shuffle=False)


print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")

loss_function = "Logloss"
task_type = 'CPU'
random_seed = 0
iterations = 300
verbose = False

params = {
    "depth": [4, 6],
    "learning_rate": [0.05, 0.1],
    "l2_leaf_reg": [3, 10],
}

model = CatBoostClassifier(
    iterations=iterations,
    loss_function=loss_function,
    task_type=task_type,
    random_seed=random_seed,
    verbose=verbose
)

cv = GridSearchCV(
    estimator=model,
    param_grid=params,
    scoring="roc_auc",
    cv=2,
    n_jobs=-1
)

clf = cv.fit(X_train, y_train)

Размер выборки для обучения: (5634, 3)
Размер выборки для теста: (1409, 3)


In [7]:
from sklearn.metrics import (
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss
)

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

cv_results = pd.DataFrame(clf.cv_results_)

best_params = clf.best_params_

model_best = CatBoostClassifier(
    iterations=iterations,
    loss_function=loss_function,
    task_type=task_type,
    random_seed=random_seed,
    verbose=verbose,
    **best_params
)

model_best.fit(X_train, y_train)

prediction = model_best.predict(X_test)
probas = model_best.predict_proba(X_test)[:, 1]

# расчёт метрик качества
metrics = {}

_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel()
auc = roc_auc_score(y_test, probas) # площадь под ROC-кривой
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

# сохранение метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

# дополнительные метрики из результатов кросс-валидации

metrics['mean_fit_time'] = cv_results['mean_fit_time'].mean()
metrics['std_fit_time'] = cv_results['std_fit_time'].mean()
metrics["mean_test_score"] = cv_results["mean_test_score"].mean()
metrics['std_test_score'] = cv_results['std_test_score'].mean()
metrics['best_score'] = clf.best_score_

In [8]:
pip_requirements= "./requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.log_params(best_params)

    mlflow.log_param("iterations", iterations)
    mlflow.log_param("loss_function", loss_function)
    mlflow.log_param("task_type", task_type)
    mlflow.log_param("random_seed", random_seed)
    
    mlflow.log_metrics(metrics)
    cv_info = mlflow.sklearn.log_model(cv, artifact_path='cv')
    
    model_info = mlflow.catboost.log_model(
        cb_model=model_best,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        pip_requirements=pip_requirements
    )
    
    run_id = run.info.run_id
    print("RUN ID:", run_id)

/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Successfully registered model 'hp_tuning_model'.
2026/02/12 09:39:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds 

RUN ID: 86c00e507fc8482694bb0543a31231f3


Created version '1' of model 'hp_tuning_model'.


In [16]:
from scipy.stats import randint, uniform
EXPERIMENT_NAME = "churn_nikolaistepanov_myown"
RUN_NAME = 'model_random_search'
REGISTRY_MODEL_NAME = "hp_tuning_model_random"

features = ["monthly_charges", "total_charges", "senior_citizen"]
target = "target"

split_column = "begin_date"
stratify_column = "target"
test_size = 0.2

df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=test_size, shuffle=False)

print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")

loss_function = "Logloss"
task_type = 'CPU'
random_seed = 0
iterations = 300
verbose = False

param_distributions = {
    "depth": randint(3, 10),
    "learning_rate": uniform(0.01, 0.29),
    "l2_leaf_reg": randint(1, 20),
    "random_strength": uniform(0, 2),
    "bagging_temperature": uniform(0, 1),
    "min_data_in_leaf": randint(1, 50)
}

model = CatBoostClassifier(
    iterations=iterations,
    loss_function=loss_function,
    task_type=task_type,
    random_seed=random_seed,
    verbose=verbose
)

cv = RandomizedSearchCV(
    estimator=model,
    n_iter=20,
    param_distributions=param_distributions,
    scoring="roc_auc",
    cv=2,
    n_jobs=-1
)

clf = cv.fit(X_train, y_train)


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

cv_results = pd.DataFrame(clf.cv_results_)

best_params = clf.best_params_

model = CatBoostClassifier(
    iterations=iterations,
    loss_function=loss_function,
    task_type=task_type,
    random_seed=random_seed,
    verbose=verbose,
    **best_params
)

model.fit(X_train, y_train)

prediction = model.predict(X_test)
probas = model.predict_proba(X_test)[:, 1]

# расчёт метрик качества
metrics = {}

_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel()
auc = roc_auc_score(y_test, probas) # площадь под ROC-кривой
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

# сохранение метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

# дополнительные метрики из результатов кросс-валидации

metrics['mean_fit_time'] = cv_results['mean_fit_time'].mean()
metrics['std_fit_time'] = cv_results['std_fit_time'].mean()
metrics["mean_test_score"] = cv_results["mean_test_score"].mean()
metrics['std_test_score'] = cv_results['std_test_score'].mean()
metrics['best_score'] = clf.best_score_

pip_requirements= "./requirements.txt"# файл с зависимостями
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.log_params(best_params)

    mlflow.log_param("iterations", iterations)
    mlflow.log_param("loss_function", loss_function)
    mlflow.log_param("task_type", task_type)
    mlflow.log_param("random_seed", random_seed)
    
    mlflow.log_metrics(metrics)
    cv_info = mlflow.sklearn.log_model(cv, artifact_path='cv')
    
    model_info = mlflow.catboost.log_model(
        cb_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        pip_requirements=pip_requirements
    )
    
    run_id = run.info.run_id
    print("RUN ID:", run_id)

Размер выборки для обучения: (5634, 3)
Размер выборки для теста: (1409, 3)


/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Successfully registered model 'hp_tuning_model_random'.
2026/02/12 09:50:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 s

RUN ID: 352730e373bb45888fb19c70e64fbc0a


Created version '1' of model 'hp_tuning_model_random'.


In [1]:
import os
import psycopg
import pandas as pd
import mlflow
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold

import optuna
from optuna.samplers import CmaEsSampler
from optuna.integration.mlflow import MLflowCallback
from collections import defaultdict
import numpy as np


connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.environ["DB_DESTINATION_HOST"],
    "port": os.environ["DB_DESTINATION_PORT"],
    "dbname": os.environ["DB_DESTINATION_NAME"],
    "user": os.environ["DB_DESTINATION_USER"],
    "password": os.environ["DB_DESTINATION_PASSWORD"],
}

connection.update(postgres_credentials)


TABLE_NAME = "users_churn"
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "churn_nikolaistepanov_myown"
RUN_NAME = "feature_selection"
REGISTRY_MODEL_NAME = "hp_tuning_model" 
FS_ASSETS = "fs_assets"


with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

yes_no_map = {"Yes": 1, "No": 0}

for col in df.columns:
    if df[col].dtype == "object":
        unique_vals = set(df[col].dropna().unique())
        if unique_vals.issubset({"Yes", "No"}):
            df[col] = df[col].map(yes_no_map)


df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,17,8191-XWSZG,2015-10-01,NaT,One year,0,Mailed check,20.65,1022.95,None,...,NaN,NaN,NaN,NaN,Female,0,0,0,0.0,0
1,59,3957-SQXML,2017-04-01,NaT,Two year,0,Credit card (automatic),24.95,894.30,None,...,NaN,NaN,NaN,NaN,Female,0,1,1,1.0,0
2,147,6837-BJYDQ,2019-11-01,NaT,One year,0,Mailed check,19.60,61.35,None,...,NaN,NaN,NaN,NaN,Male,0,0,0,0.0,0
3,481,0486-LGCCH,2019-03-01,NaT,Two year,0,Mailed check,19.65,225.75,None,...,NaN,NaN,NaN,NaN,Male,0,1,1,0.0,0
4,1001,8357-EQXFO,2019-04-01,2019-11-01,Month-to-month,1,Electronic check,95.35,660.90,Fiber optic,...,1.0,0.0,1.0,1.0,Female,0,0,0,0.0,1


In [2]:
features = ["monthly_charges", "total_charges", "senior_citizen"]
target = "target"

split_column = "begin_date"
stratify_column = "target"
test_size = 0.2

df = df.sort_values(by=[split_column])

X_train, val_X, y_train, val_y = train_test_split(df[features], df[target], test_size=test_size, shuffle=False)


In [3]:
from numpy import array, median
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from sklearn.metrics import (
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss
)


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

EXPERIMENT_NAME = "optuna_hyperparameter_tuning_2"
RUN_NAME = "model_bayesian_search"

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"


def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
 
    model = CatBoostClassifier(**param)

    skf = StratifiedKFold(n_splits=2)

    metrics = defaultdict(list)
    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        train_x = X_train.iloc[train_index]
        val_x = X_train.iloc[val_index]

        train_y = y_train.iloc[train_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(train_x, train_y)
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]

        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        auc = roc_auc_score(val_y, probas)
        precision = precision_score(val_y, prediction)
        recall = recall_score(val_y, prediction)
        f1 = f1_score(val_y, prediction)
        logloss = log_loss(val_y, prediction)
        
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["auc"].append(auc)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["f1"].append(f1)
        metrics["logloss"].append(logloss)


    # ваш код здесь #
    err_1 = median(array(metrics['err1']))
    err_2 = median(array(metrics['err2']))
    auc = median(array(metrics['auc']))
    precision = median(array(metrics['precision']))
    recall = median(array(metrics['recall']))
    f1 = median(array(metrics['f1']))
    logloss = median(array(metrics['logloss']))
		

    return auc


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id
    

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

mlflc = MLflowCallback(
    metric_name="AUC",
    tracking_uri=f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}',
    create_experiment=False,
    mlflow_kwargs={'experiment_id': experiment_id, 'tags': {MLFLOW_PARENT_RUN_ID: run_id}}
   )

study = optuna.create_study(direction="maximize", study_name=STUDY_NAME, load_if_exists=True, sampler=optuna.samplers.TPESampler(), storage=STUDY_DB_NAME)
study.optimize(objective, n_trials=10, callbacks=[mlflc])
best_params = study.best_params
print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")

/tmp/ipykernel_16758/2899914819.py:91: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2026-02-13 17:09:45,934] Using an existing study with name 'churn_model' instead of creating a new one.
/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2026-02-13 17:09:46,762] Trial 43 finished with value: 0.8176747279664972 and parameters: {'learning_rate': 0.03675588387635633, 'depth': 2, 'l2_leaf_reg': 2.184612997728701, 'random_strength': 4.449821002030486}. Best is trial 1 with value: 0.8258245883578705.
/home/mle-user/mle_projects/mle-mlflow/.venv_mle_mlflow/lib/python3.10/site-packages/sklearn/metrics/_classifi

Number of finished trials: 53
Best params: {'learning_rate': 0.024506580722380395, 'depth': 4, 'l2_leaf_reg': 1.947133857850558, 'random_strength': 4.79773616047571}


In [4]:
best_model = CatBoostClassifier(
    **best_params,
    loss_function="Logloss",
    task_type="CPU",
    random_seed=0,
    iterations=300,
    verbose=False,
)

best_model.fit(X_train, y_train)

with mlflow.start_run(run_id=run_id):
    mlflow.catboost.log_model(
        best_model,
        artifact_path="cv",
        registered_model_name=REGISTRY_MODEL_NAME
    )



Registered model 'hp_tuning_model' already exists. Creating a new version of this model...
2026/02/13 17:14:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: hp_tuning_model, version 2
Created version '2' of model 'hp_tuning_model'.


2.0